https://boxil.jp/service/197/reviews/

In [ ]:
# import
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import warnings
import streamlit as st
from keras.layers import Dense, Activation
from keras.models import Sequential
from tensorflow import keras
from sklearn.cluster import KMeans
import warnings

In [ ]:
df = pd.read_csv('customer_review.csv')

## data cleaning

In [3]:
df.drop_duplicates(inplace = True)

In [4]:
df.isna().sum()

タイトル                                                                      1
利用状況                                                                      1
reputationAnswerItemTopRightItemBlock__typeOfBusiness                     3
投稿日                                                                       3
reputationAnswerItemStatusScoreBlock__scoreNum                            1
reputationAnswerItemBasicBlock__description                               1
利用アカウント数                                                                  1
reputationAnswerItemFirstQuestionBlock__answer                            1
reputationAnswerSectionFunctionRateQuestionBlock__answer__rateNumber      1
reputationAnswerSectionFunctionRateQuestionBlock__answer__rateNumber1     1
reputationAnswerSectionFunctionRateQuestionBlock__answer__rateNumber2     1
reputationAnswerSectionFunctionRateQuestionBlock__answer__rateNumber3     2
reputationAnswerSectionFunctionRateQuestionBlock__answer__rateNumber4    17
dtype: int64

In [5]:
# df.drop(columns = [])

In [32]:
# change col names
df.columns = ['topic', 'usage situation', 'type of business', 'review date', 'overall rating', 'review', 
             'number of accounts used', 
             'effects after introducing the service', 'rate ease of use', 'rate helpfulness', 'rate customizability',
              'rate functional satisfaction', 'rate service stability']

In [7]:
df.columns

Index(['topic', 'usage situation', 'type of business', 'review date',
       'overall rating', 'review', 'number of accounts used',
       'Effects after introducing the service', 'rate ease of use',
       'rate helpfulness', 'rate customizability',
       'rate functional satisfaction', 'rate service stability'],
      dtype='object')

In [8]:
df.drop([24,25], axis=0, inplace=True)

In [9]:
df[df['type of business'].isna()]

,topic,usage situation,type of business,review date,overall rating,review,number of accounts used,Effects after introducing the service,rate ease of use,rate helpfulness,rate customizability,rate functional satisfaction,rate service stability
22,営業情報共有ツールの定番,利用中,NaN,NaN,4/5,社員が多い当社においては得意先アプローチ情報の共有が容易ではなく、このような情報共有ツールは...,1000件以上,10000人近く所属している社員同士の情報共有が容易にできる。アプローチしたい得意先に関する...,3.0,3.0,4.0,4.0,4.0
185,わかりやすいサービスでした！,現在は利用していない,NaN,NaN,4/5,自分の能力もそうでしたが、とても働きやすい環境ではありました。またいろんなサービスを利用でき...,1件,タスクの管理が難しく感じました。スケジュールが立てづらいのもありましたが、そこは仕方なかった...,5.0,5.0,5.0,5.0,4.0


In [10]:
df['type of business'].value_counts()
# fill na with mode
df['type of business'] = df['type of business'].fillna(df['type of business'].mode()[0])
df['review date'] = df['review date'].fillna(df['review date'].mode()[0])

In [11]:
df['rate service stability'].value_counts()
print('mean of rate of service stability: ', (df['rate service stability'].mean()))
## fill with 4 
df['rate service stability'] = df['rate service stability'].fillna(4.0)
df[df['rate functional satisfaction'].isna()]
## fill with 2
df['rate functional satisfaction'] = df['rate functional satisfaction'].fillna(2.0)

mean of rate of service stability:  4.266666666666667


In [12]:
df.isna().sum()

topic                                    0
usage situation                          0
type of business                         0
review date                              0
overall rating                           0
review                                   0
number of accounts used                  0
Effects after introducing the service    0
rate ease of use                         0
rate helpfulness                         0
rate customizability                     0
rate functional satisfaction             0
rate service stability                   0
dtype: int64

In [13]:
df.loc[df['overall rating'].str.contains('1', na=False, regex=True), 'overall rating'] = 1
df.loc[df['overall rating'].str.contains('2', na=False, regex=True), 'overall rating'] = 2
df.loc[df['overall rating'].str.contains('3', na=False, regex=True), 'overall rating'] = 3
df.loc[df['overall rating'].str.contains('4', na=False, regex=True), 'overall rating'] = 4
df.loc[df['overall rating'].str.contains('5', na=False, regex=True), 'overall rating'] = 5

In [14]:
df['overall rating'].apply(pd.to_numeric)

0      4
1      5
2      3
3      4
4      5
      ..
247    5
248    4
249    4
250    5
251    4
Name: overall rating, Length: 241, dtype: int64

In [18]:
df.loc[df['number of accounts used'].str.contains('以上', na=False, regex=True), 'number of accounts used'] = '>1000'
df.loc[df['number of accounts used'].str.contains('〜1000件', na=False, regex=True), 'number of accounts used'] = '501-1000'
df.loc[df['number of accounts used'].str.contains('〜500件', na=False, regex=True), 'number of accounts used'] = '301-500'
df.loc[df['number of accounts used'].str.contains('〜300件', na=False, regex=True), 'number of accounts used'] = '>201-300'
df.loc[df['number of accounts used'].str.contains('〜200件', na=False, regex=True), 'number of accounts used'] = '>101-200'
df.loc[df['number of accounts used'].str.contains('〜100件', na=False, regex=True), 'number of accounts used'] = '>51-100'
df.loc[df['number of accounts used'].str.contains('〜50件', na=False, regex=True), 'number of accounts used'] = '>31-50'
df.loc[df['number of accounts used'].str.contains('〜30件', na=False, regex=True), 'number of accounts used'] = '>11-30'
df.loc[df['number of accounts used'].str.contains('〜10件', na=False, regex=True), 'number of accounts used'] = '2-10'
df.loc[df['number of accounts used'].str.contains('1件', na=False, regex=True), 'number of accounts used'] = '>1'

In [20]:
df['number of accounts used'].nunique()

10

In [23]:
df.loc[df['usage situation'].str.contains('利用中', na=False, regex=True), 'usage situation'] = 'using'
df.loc[df['usage situation'].str.contains('していない', na=False, regex=True), 'usage situation'] = 'currently not using'

In [24]:
df['usage situation'].unique()

array(['using', 'currently not using'], dtype=object)

In [40]:
df.head()

,topic,usage situation,type of business,review date,overall rating,review,number of accounts used,effects after introducing the service,rate ease of use,rate helpfulness,rate customizability,rate functional satisfaction,rate service stability
0,カスタマーサポートで大活躍,using,Consulting Service,2022-07-28,4,顧客ごとで以前のやり取りの履歴が管理できたり、進行状況なども一括管理することができる。\n複...,>201-300,以前は営業管理や人事管理が曖昧で生産性が下がっていたが、サービス導入によりシステムで一元管理...,4.0,4.0,4.0,4.0,4.0
1,顧客管理を一つのアプリで共有できます。,using,IT related Service,2022-07-28,5,社内の顧客情報をSalesforceにまとめて管理することで部署を超えた業務のコラボレーショ...,>51-100,営業活動のデータにはそれぞれの顧客との会話、反応や特徴を書き込むことができ、蓄積されるにつれ...,4.0,4.0,5.0,5.0,4.0
2,導入時のコンサルに時間をかけないと使い勝手が悪くなる,using,Food and Leisure Service,2022-07-22,3,導入時のコンサルが成功のカギ。時間をかけないで曖昧な部分を残すと、使い勝手の悪いものになる。...,>51-100,事業部ごとにバラバラに管理していたデータを集約できたのはとても良かった。事業部ごとのそれぞれ...,2.0,3.0,2.0,2.0,5.0
3,顧客管理が簡単に,using,IT related Service,2022-07-20,4,このサービスは顧客の情報を細かく管理することができ、それを簡単に閲覧できるので、営業の質のア...,>1,顧客管理を紙で行っていたのですが、クラウドで行えるようになったので、データ消失の心配がなくな...,2.0,4.0,3.0,4.0,3.0
4,部門間の連をがより強固にすることが可能,using,Manufacturer/manufacturing system,2022-07-19,5,Salesforce Sales Cloudはとても見やすいUIを有しているので、プロジェク...,2-10,Salesforce Sales Cloudを使うことによって、各営業担当に進歩状況を都度メ...,4.0,5.0,3.0,4.0,3.0


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 251
Data columns (total 13 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   topic                                  241 non-null    object        
 1   usage situation                        241 non-null    object        
 2   type of business                       241 non-null    object        
 3   review date                            241 non-null    datetime64[ns]
 4   overall rating                         241 non-null    object        
 5   review                                 241 non-null    object        
 6   number of accounts used                241 non-null    object        
 7   Effects after introducing the service  241 non-null    object        
 8   rate ease of use                       241 non-null    float64       
 9   rate helpfulness                       241 non-null    float64   

In [28]:
df['review date'] =  pd.to_datetime(df['review date'], infer_datetime_format=True)

In [36]:
df.loc[df['type of business'].str.contains('コンサル', na=False, regex=True), 'type of business'] = 'Consulting Service'
df.loc[df['type of business'].str.contains('IT', na=False, regex=True), 'type of business'] = 'IT related Service'
df.loc[df['type of business'].str.contains('外食', na=False, regex=True), 'type of business'] = 'Food and Leisure Service'
df.loc[df['type of business'].str.contains('メーカー', na=False, regex=True), 'type of business'] = 'Manufacturer/manufacturing system'
df.loc[df['type of business'].str.contains('不動産', na=False, regex=True), 'type of business'] = 'Real Estate/Construction/Equipment'
df.loc[df['type of business'].str.contains('金融', na=False, regex=True), 'type of business'] = 'Finance/Insurance'
df.loc[df['type of business'].str.contains('マスコミ', na=False, regex=True), 'type of business'] = 'Mass Media/Advertising/Design/Game/Entertainment'
df.loc[df['type of business'].str.contains('輸送', na=False, regex=True), 'type of business'] = 'Transportation/Traffic/Logistics/Warehouse system'
df.loc[df['type of business'].str.contains('小売', na=False, regex=True), 'type of business'] = 'Retail/distribution/trading companies'
df.loc[df['type of business'].str.contains('医療系', na=False, regex=True), 'type of business'] = 'Medical'


In [38]:
df['type of business'].nunique()

10

In [39]:
df.to_csv('data_cleaned.csv')

In [41]:
from googletrans import Translator
tr = Translator(service_urls=['translate.googleapis.com'])
tr = Translator()


df['topic'] = df['topic'].apply(lambda x : tr.translate(x, src="ja", dest="en").text)
df['review'] = df['review'].apply(lambda x : tr.translate(x, src="ja", dest="en").text)
df['effects after introducing the service'] = df['effects after introducing the service'].apply(lambda x : tr.translate(x, src="ja", dest="en").text)


In [43]:
df.to_csv('data_cleaned_translated.csv')